In [42]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import folium

Download the data from the USPS. Save the data as a CSV on your local system.

In [30]:
headers = {"User-Agent" : "User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;"}
url = 'https://about.usps.com/who/legal/foia/documents/change-of-address-stats/Y2020.csv'
response = requests.get(url,headers=headers)

In [31]:
with open(r'USPS_Moves2020.csv', 'wb') as f:
    f.write(response.content)

Load the csv as a dataframe. You will need to wait for the previous cell to complete running before this cell will work.

In [32]:
df = pd.read_csv('USPS_Moves2020.csv')
df.head()

,YYYYMM,ZIPCODE,CITY,STATE,TOTAL FROM ZIP,TOTAL BUSINESS,TOTAL FAMILY,TOTAL INDIVIDUAL,TOTAL PERM,TOTAL TEMP,TOTAL TO ZIP,TOTAL BUSINESS.1,TOTAL FAMILY.1,TOTAL INDIVIDUAL.1,TOTAL PERM.1,TOTAL TEMP.1
0,202001,"=""00601""",ADJUNTAS,PR,13,0,0,0,13,0,0,0,0,0,0,0
1,202001,"=""00602""",AGUADA,PR,25,0,0,16,25,0,40,0,0,27,34,0
2,202001,"=""00603""",AGUADILLA,PR,45,0,15,27,40,0,63,0,15,45,49,14
3,202001,"=""00605""",AGUADILLA,PR,13,0,0,0,12,0,18,0,0,12,17,0
4,202001,"=""00610""",ANASCO,PR,20,0,0,20,19,0,21,0,0,18,17,0


Select the data that corresponds to Memphis, TN. The city field has a weird formating.

In [33]:
mem = df[(df['CITY'] == 'MEMPHIS                     ') & (df['STATE']== 'TN')]
mem.head()

,YYYYMM,ZIPCODE,CITY,STATE,TOTAL FROM ZIP,TOTAL BUSINESS,TOTAL FAMILY,TOTAL INDIVIDUAL,TOTAL PERM,TOTAL TEMP,TOTAL TO ZIP,TOTAL BUSINESS.1,TOTAL FAMILY.1,TOTAL INDIVIDUAL.1,TOTAL PERM.1,TOTAL TEMP.1
9856,202001,"=""38101""",MEMPHIS,TN,11,0,0,0,11,0,46,0,18,27,44,0
9857,202001,"=""38103""",MEMPHIS,TN,199,0,37,153,187,12,171,12,21,138,154,17
9858,202001,"=""38104""",MEMPHIS,TN,275,19,42,214,260,15,223,0,38,179,209,14
9859,202001,"=""38105""",MEMPHIS,TN,82,0,0,70,72,0,69,0,11,58,61,0
9860,202001,"=""38106""",MEMPHIS,TN,256,0,74,179,237,19,246,0,75,168,226,20


Using NumPy, group by the ZIPCODE field. Summate the to family and individual to and from moves.

In [34]:
zip = mem.groupby('ZIPCODE').agg({'TOTAL FAMILY': np.sum, 'TOTAL INDIVIDUAL': np.sum, 'TOTAL FAMILY.1': np.sum, 'TOTAL INDIVIDUAL.1': np.sum })
print(zip)

          TOTAL FAMILY  TOTAL INDIVIDUAL  TOTAL FAMILY.1  TOTAL INDIVIDUAL.1
ZIPCODE                                                                     
="38101"             0                11              46                 219
="38103"           590              2764             404                2286
="38104"           607              2807             469                2479
="38105"           151               761              51                 542
="38106"           685              1771             655                1677
="38107"           513              1414             422                1277
="38108"           472              1166             471                1026
="38109"          1011              2612            1096                2668
="38111"          1134              3444             917                2973
="38112"           414              1245             342                1157
="38114"           809              2135             709                1860

Create two new fields, movefrom and moveto, which are summations of the individual and family moves.

In [35]:
movefrom = zip['TOTAL FAMILY'] + zip['TOTAL INDIVIDUAL']
moveto = zip['TOTAL FAMILY.1'] + zip['TOTAL INDIVIDUAL.1']
zip['movefrom'] = movefrom
zip['moveto'] = moveto
zip.head()

,TOTAL FAMILY,TOTAL INDIVIDUAL,TOTAL FAMILY.1,TOTAL INDIVIDUAL.1,movefrom,moveto
ZIPCODE,,,,,,
"=""38101""",0,11,46,219,11,265
"=""38103""",590,2764,404,2286,3354,2690
"=""38104""",607,2807,469,2479,3414,2948
"=""38105""",151,761,51,542,912,593
"=""38106""",685,1771,655,1677,2456,2332


Create another new field, netmoves, which is moveto – movefrom. 

In [36]:
netmoves = zip['moveto'] - zip['movefrom']
zip['netmoves'] = netmoves
zip.head()

,TOTAL FAMILY,TOTAL INDIVIDUAL,TOTAL FAMILY.1,TOTAL INDIVIDUAL.1,movefrom,moveto,netmoves
ZIPCODE,,,,,,,
"=""38101""",0,11,46,219,11,265,254
"=""38103""",590,2764,404,2286,3354,2690,-664
"=""38104""",607,2807,469,2479,3414,2948,-466
"=""38105""",151,761,51,542,912,593,-319
"=""38106""",685,1771,655,1677,2456,2332,-124


The ZIPCODE is an index in the dataframe. Create a new field, ZIP, which is equal to the value of the index for each record. 

In [37]:
zip['ZIP'] = zip.index
zip.head()

,TOTAL FAMILY,TOTAL INDIVIDUAL,TOTAL FAMILY.1,TOTAL INDIVIDUAL.1,movefrom,moveto,netmoves,ZIP
ZIPCODE,,,,,,,,
"=""38101""",0,11,46,219,11,265,254,"=""38101"""
"=""38103""",590,2764,404,2286,3354,2690,-664,"=""38103"""
"=""38104""",607,2807,469,2479,3414,2948,-466,"=""38104"""
"=""38105""",151,761,51,542,912,593,-319,"=""38105"""
"=""38106""",685,1771,655,1677,2456,2332,-124,"=""38106"""


Clean the ZIP field with a regular expression.

In [40]:
zip['name'] = zip['ZIP'].str.extract('(\d+)')
#The code below can be used if you need to convert the field to a numeric type for joining later.
#zip['name'] = pd.to_numeric(zip['name'])
zip.head()

,TOTAL FAMILY,TOTAL INDIVIDUAL,TOTAL FAMILY.1,TOTAL INDIVIDUAL.1,movefrom,moveto,netmoves,ZIP,name
ZIPCODE,,,,,,,,,
"=""38101""",0,11,46,219,11,265,254,"=""38101""",38101
"=""38103""",590,2764,404,2286,3354,2690,-664,"=""38103""",38103
"=""38104""",607,2807,469,2479,3414,2948,-466,"=""38104""",38104
"=""38105""",151,761,51,542,912,593,-319,"=""38105""",38105
"=""38106""",685,1771,655,1677,2456,2332,-124,"=""38106""",38106


Load the shapefile of Memphis’ ZIP codes using Geopandas. The file is stored on the local machine.

In [41]:
fname = 'data/Zip_Codes.shp'
geo = gpd.read_file(fname)
geo.head()

,name,shape_area,geometry
0,38018,10.068270,"MULTIPOLYGON (((-89.80634 35.15960, -89.80636 ..."
1,38053,0.033496,"MULTIPOLYGON (((-89.97518 35.25242, -89.97413 ..."
2,38103,4.604346,"POLYGON ((-90.04591 35.18828, -90.04578 35.187..."
3,38104,4.902973,"POLYGON ((-90.00875 35.15485, -90.00893 35.153..."
4,38105,1.624652,"POLYGON ((-90.01855 35.15551, -90.01902 35.152..."


Join the shapefile and the dataframe on the name fields.

In [44]:
geo = geo.merge(zip, on = 'name')
geo.head()

,name,shape_area,geometry,TOTAL FAMILY,TOTAL INDIVIDUAL,TOTAL FAMILY.1,TOTAL INDIVIDUAL.1,movefrom,moveto,netmoves,ZIP
0,38103,4.604346,"POLYGON ((-90.04591 35.18828, -90.04578 35.187...",590,2764,404,2286,3354,2690,-664,"=""38103"""
1,38104,4.902973,"POLYGON ((-90.00875 35.15485, -90.00893 35.153...",607,2807,469,2479,3414,2948,-466,"=""38104"""
2,38105,1.624652,"POLYGON ((-90.01855 35.15551, -90.01902 35.152...",151,761,51,542,912,593,-319,"=""38105"""
3,38106,14.572111,"POLYGON ((-90.05838 35.12699, -90.05841 35.126...",685,1771,655,1677,2456,2332,-124,"=""38106"""
4,38107,5.543473,"MULTIPOLYGON (((-90.03860 35.18995, -90.03738 ...",513,1414,422,1277,1927,1699,-228,"=""38107"""


Create a web map and overlay the data using a choropleth.

In [48]:
memmap = folium.Map(location=[35.1, -89.95], zoom_start=11)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(memmap)

myscale = (geo['netmoves'].quantile((0,0.25,0.5,0.75,1))).tolist()

folium.Choropleth(geo,                                # geo data
                  data=geo,                           # data
                  key_on='feature.properties.name', # feature.properties.key
                  columns=['name', 'netmoves'],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  threshold_scale=myscale,
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  legend_name='Net Moves').add_to(memmap)

folium.LayerControl().add_to(memmap)

memmap